# Colab

In [ ]:
!git clone https://github.com/Tio-Panda/inf577-project.git

# Init

In [ ]:
import h5py
from tqdm import tqdm

from modules import ReconstructionRF, get_grid, get_bmode, save_bmode, DataLoader

%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import tensorflow as tf
from pathlib import Path

base_path = "." # local
#base_path = "/content/inf557-project" # colab

sys.path.append(f"{base_path}/modules")

# Create dataset

In [ ]:
Nz = 2048
Nx = 256
new_Ns = 2800

dl = DataLoader(f"{base_path}/original_dataset", "./data.csv")

df = dl.get_df()
df = df.query("RF == 1 and n_channels == 128")
df = df[df["name"].str[:3] != "JHU"]
df = df[df["source"].str[:3] != "PICMUS"]
names = df["name"]

pbar = tqdm(names, desc="Files", unit="file")

stages = [
    "load files",
    "padding",
    "beamforming",
    "save"
]

for name in pbar:

    pbar.set_description(f"-> {name}")

    pw = dl.get_defined_pwdata(selected_name=name, mode="RF")

    if pw.n_samples <= new_Ns:
        pw.rfdata = np.pad(pw.rfdata, pad_width=((0,0), (0,0), (0, new_Ns - pw.n_samples)), mode='constant', constant_values=0)
    else:
        pw.rfdata = pw.rfdata[:, :, :new_Ns]
        pw.img_depth = (new_Ns / pw.fs) * (pw.c0 / 2)
    pw.n_samples = new_Ns

    grid = get_grid(pw, Nz, Nx)

    rf_max = pw.rfdata / np.max(np.abs(pw.rfdata))
    sigma = np.std(rf_max)
    pw.rfdata = rf_max / sigma

    engine = ReconstructionRF(pw, grid)
    selected_angles = np.arange(pw.n_angles)

    imgs = engine.mv_bmfrm(selected_angles, L=32, delta=0.05, z_chunk=800)
    img = imgs.mean(axis=0)
    bimg = get_bmode(img)

    with h5py.File(base_path+"/processed_dataset/"+name+".hdf5", "w") as f:
        f.create_dataset("img", data=img, compression="gzip", chunks=True)
        f.create_dataset("rfdata", data=pw.rfdata[pw.n_angles//2, ...], compression="gzip", chunks=True)
        f.create_dataset("grid", data=grid, compression="gzip", chunks=True)
        f.create_dataset("probe_geometry", data=pw.probe_geometry, compression="gzip", chunks=True)

        f.attrs["zlims"] = pw.zlims
        f.attrs["fs"] = np.squeeze(pw.fs)
        f.attrs["c0"] = np.squeeze(pw.c0)
        f.attrs["t0"] = np.squeeze(pw.t0[pw.n_angles//2])
        f.attrs["angle"] = np.squeeze(pw.angles[pw.n_angles//2])
        f.attrs["Ns"] = pw.n_samples
        f.attrs["Nc"] = pw.n_channels
        f.attrs["Nz"] = Nz
        f.attrs["Nx"] = Nx
        f.attrs["name"] = pw.name